In [20]:
import mido
import os
import pandas as pd
import numpy as np
from aaapi import MidiUtils

In [21]:
midi_dir = '../dataset/midi_by_tracks'
midi_files = os.listdir(midi_dir)

In [22]:
def is_valid(track):
    expecting_on = True
    at_time = 0
    for msg in track:
        at_time += msg.time
        if 'note' in msg.type:
            if (expecting_on and msg.type == 'note_on') or (not expecting_on and msg.type == 'note_off'):
                expecting_on = not expecting_on
            else:
                return False, at_time
    return True, None

In [23]:
def is_all_note_on(track):
    for msg in track:
        if 'note' in msg.type and 'off' in msg.type:
            return False
    return True

In [24]:
def vel0_to_note_off(track):
    nt = list()
    for msg in track:
        if msg.type == 'note_on' and msg.velocity == 0:
            msg = mido.Message(
                type='note_off', 
                channel=msg.channel,
                note=msg.note,
                velocity=msg.velocity,
                time=msg.time
            )
        nt += [msg]
    return nt

In [25]:
def sanitise(track):
    added_rest = 0
    nt = list()
    for msg in track:
        if 'note' in msg.type:
            msg.time += added_rest
            added_rest = 0
            nt += [msg]
        else:
            added_rest += msg.time
    return nt

In [17]:
raise Exception()
for f in midi_files:
    fpath = os.path.join(midi_dir, f)
    fpath = os.path.join(fpath, os.listdir(fpath)[0])
    midi = mido.MidiFile(fpath)
    midi.tracks = [sanitise(midi.tracks[0])]
    midi.save(fpath)
    print("Sanitized", f)

Exception: 

In [53]:
for f in midi_files:
    fpath = os.path.join(midi_dir, f)
    fpath = os.path.join(fpath, os.listdir(fpath)[0])
    midi = mido.MidiFile(fpath)
    t = midi.tracks[0]
    
    mdf = list()
    for msg in t:
        ntype = msg.type
        ntime = msg.time
        mdf += [[ntype, ntime]]
    mdf = pd.DataFrame(mdf, columns=['type', 'time'])

    valid, time = is_valid(t)
    if not valid:
        if is_all_note_on(t):
            midi.tracks = [vel0_to_note_off(t)]
            midi.save(fpath)
            print("Converted", f, "to note_on and note_off messages")        

Converted taylor_swift-safe_and_sound_feat_the_civil_wars.mid to note_on and note_off messages
Converted bruno_mars-locked_out_of_heaven.mid to note_on and note_off messages
Converted demi_lovato-skyscraper.mid to note_on and note_off messages
Converted taylor_swift-i_knew_you_were_trouble.mid to note_on and note_off messages
Converted Backstreet Boys - I Want It That Way.mid to note_on and note_off messages
Converted Westlife_Swear_It_Again.mid to note_on and note_off messages
Converted avril_lavigne-when_youre_gone.mid to note_on and note_off messages
Converted avril_lavigne-unwanted.mid to note_on and note_off messages
Converted WESTLIFE.I lay my love on you.mid to note_on and note_off messages
Converted bruno_mars-just_the_way_you_are.mid to note_on and note_off messages
Converted ed_sheeran-thinking_out_loud.mid to note_on and note_off messages
Converted GREENDAY.In the end.mid to note_on and note_off messages
Converted Britney_Spears_-_You_Drive_Me_Crazy.mid to note_on and note_o

In [87]:
for f in midi_files:
    fpath = os.path.join(midi_dir, f)
    fpath = os.path.join(fpath, os.listdir(fpath)[0])
    midi = mido.MidiFile(fpath)
    t = midi.tracks[0]
    
    mdf = list()
    for msg in t:
        ntype = msg.type
        ntime = msg.time
        mdf += [[ntype, ntime]]
    mdf = pd.DataFrame(mdf, columns=['type', 'time'])

    valid, time = is_valid(t)
    if not valid:
        print(f, "at time", time)     
        

bruno_mars-when_i_was_your_man.mid at time 120191


In [50]:
vel0_to_note_off(midi.tracks[0])

[<message note_on channel=3 note=61 velocity=120 time=2879>,
 <message note_off channel=3 note=61 velocity=0 time=400>,
 <message note_on channel=3 note=58 velocity=119 time=16>,
 <message note_off channel=3 note=58 velocity=0 time=119>,
 <message note_on channel=3 note=58 velocity=98 time=29>,
 <message note_off channel=3 note=58 velocity=0 time=188>,
 <message note_on channel=3 note=56 velocity=119 time=13>,
 <message note_off channel=3 note=56 velocity=0 time=431>,
 <message note_on channel=3 note=54 velocity=120 time=6>,
 <message note_off channel=3 note=54 velocity=0 time=150>,
 <message note_on channel=3 note=54 velocity=104 time=22>,
 <message note_off channel=3 note=54 velocity=0 time=165>,
 <message note_on channel=3 note=61 velocity=119 time=16>,
 <message note_off channel=3 note=61 velocity=0 time=373>,
 <message note_on channel=3 note=58 velocity=119 time=25>,
 <message note_off channel=3 note=58 velocity=0 time=312>,
 <message note_on channel=3 note=56 velocity=119 time=38

In [51]:
is_valid(midi.tracks[0])

(False, 3279)